## Imports

In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from transformers import AutoModel, AutoTokenizer, AdamW, get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup, AutoConfig



d:\Program Files\anaconda3\envs\ciscn\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## EDA

In [2]:
train_df = pd.read_csv('./train.csv')
train_df.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate


In [3]:
train_df.describe()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
count,36765,36765,36765,36765,36765
unique,36765,4191,36691,7,3
top,0013cc385424,91B1F82B2CF1,Summer projects should be student-designed,Evidence,Adequate
freq,1,23,14,12105,20977


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36765 entries, 0 to 36764
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   discourse_id             36765 non-null  object
 1   essay_id                 36765 non-null  object
 2   discourse_text           36765 non-null  object
 3   discourse_type           36765 non-null  object
 4   discourse_effectiveness  36765 non-null  object
dtypes: object(5)
memory usage: 1.4+ MB


In [5]:
train_df.discourse_effectiveness.value_counts()

Adequate       20977
Effective       9326
Ineffective     6462
Name: discourse_effectiveness, dtype: int64

In [6]:
train_df.loc[train_df['discourse_effectiveness'] == 'Effective'].head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
9,d226f06362f5,00944C693682,Limiting the usage of cars has personal and pr...,Lead,Effective
10,de347c859ab6,00944C693682,With so many things in this world that few peo...,Position,Effective
12,a6fcdd9110ab,00944C693682,It is no secret that morning traffic jams and ...,Evidence,Effective
13,6efd9102298b,00944C693682,the environment suffers greatly from the many ...,Claim,Effective
14,d6807f31da92,00944C693682,"""Passenger cars are responsible for 12 percent...",Evidence,Effective


In [7]:
(len(train_df.loc[train_df['discourse_effectiveness'] == 'Effective']),len(train_df.loc[train_df['discourse_effectiveness'] == 'Adequate']),len(train_df.loc[train_df['discourse_effectiveness'] == 'Ineffective']))

(9326, 20977, 6462)

In [11]:
train_df['discourse_text'].head()
train_df['discourse_text'].describe()

AttributeError: 'Series' object has no attribute 'info'

There is a problem that number of each type is not equal.